# Day 21
## Part 1

In [7]:
def die():
    while True:
        yield from range(1, 101)
        
def rolls():
    d = die()
    while True:
        yield next(d) + next(d) + next(d)
        
def move(p, n):
    return (p - 1 + n) % 10 + 1

def part_1(player1, player2):
    r = rolls()
    n_rolls = 0
    scores = [0, 0]
    positions = [player1, player2]
    while True:
        for i in range(2):
            positions[i] = move(positions[i], next(r))
            scores[i] += positions[i]
            n_rolls += 3
            if scores[i] >= 1000:
                return scores[0 if i else 1] * n_rolls
            
assert part_1(4, 8) == 739785

In [8]:
part_1(8, 9)

512442

## Part 2
Ha ha! I thought part 1 was a bit easy for day 21.